# TensorRT Engine Report Card

Use this Jupyter worksheet to get a quick overview of the structure and characteristics of a TensorRT Engine plan.

In [ ]:
import IPython
from ipywidgets import widgets
from trex import *
from trex.notebook import *
from trex.report_card import *

# Choose an engine file to load.
engine_name = "../tests/inputs/mobilenet.qat.onnx.engine"
engine_name = "../tests/inputs/mobilenet_v2_residuals.qat.onnx.engine"
set_wide_display()

## Load JSON Files

In [ ]:
plan = EnginePlan(f"{engine_name}.graph.json", f"{engine_name}.profile.json", f"{engine_name}.profile.metadata.json")

### Render Graph

In [ ]:
report_card_draw_plan_graph_extended(plan, engine_name)

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Plan Summary</div></html>

In [ ]:
# "Average time" refers to the sum of the layer latencies, when profiling layers separately
# "Latency" refers to the [min, max, mean, median, 99% percentile] of the engine latency measurements, when timing the engine w/o profiling layers.
plan.summary()

In [ ]:
layer_latency_sunburst(plan.df, "Layers Latencies (%)")

In [ ]:
report_card_table_view(plan)

### Timings

In [ ]:
plot_engine_timings(timing_json_file= f"{engine_name}.timing.json")

## Performance

In [ ]:
report_card_perf_overview_widget(plan)

## Memory Footprint

In [ ]:
report_card_memory_footprint_widget(plan)

## Tactics

In [ ]:
latency_vs_prec_per_conv = partial(
    plotting.plotly_bar2,
    plan.df,
    values_col='latency.pct_time',
    names_col='Name',
    color='tactic')

latency_vs_prec_per_conv("Latency per Layer (color=Tactics)")

tactic_cnt = group_count(plan.df, 'tactic')
display_df(tactic_cnt)

<html><div style="text-align:center;background:#76b900;padding:20px;color:#ffffff;font-size:2em;">Excel Summary</div></html>

### Generate a default summary

In [ ]:
from trex.excel_summary import *

summary = ExcelSummary(plan, path="default_summary.xlsx")
summary.generate_default_summary() # 'generate_default_summary' automatically saves the file

### Generate a customized summary

In [ ]:
summary = ExcelSummary(plan, path="customized_summary.xlsx")
summary.add_dataframes({"df": plan.df})
summary.add_images({"trex_logo": "../images/trex_logo.png"})
summary.add_dataframes({"clean_df": clean_for_display(plan.df)})
summary.save() # 'save' needs to be called to propagate the changes to the file

In [ ]:
# The context manager automatically saves the file
with ExcelSummary(plan, path="customized_summary_with_manager.xlsx") as summary:
    summary.add_dataframes({"df": plan.df})
    summary.add_images({"trex_logo": "../images/trex_logo.png"})
    summary.add_dataframes({"clean_df": clean_for_display(plan.df)})
